In [ ]:
import cv2
! pip install mediapipe
import mediapipe as mp
import glob
from google.colab.patches import cv2_imshow
import matplotlib.pyplot as plt
import os
import math

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 32.9 MB 171 kB/s 


In [ ]:
# Let's make sure the kaggle.json file is present.
!ls -lha kaggle.json
# Next, install the Kaggle API client.
!pip install -q kaggle
# The Kaggle API client expects this file to be in ~/.kaggle,
# so move it there.
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
# This permissions change avoids a warning on Kaggle tool startup.
!chmod 600 ~/.kaggle/kaggle.json

# Copy the stackoverflow data set locally.
!kaggle datasets download -d niten19/face-shape-dataset

#Zip the files
!unzip -q face-shape-dataset.zip
#Search for file's locations
! ls -d $PWD/*

-rw-r--r-- 1 root root 63 Aug 22 03:42 kaggle.json
 99% 632M/639M [00:03<00:00, 226MB/s]
100% 639M/639M [00:03<00:00, 200MB/s]
 /content/drive			   /content/kaggle.json
'/content/FaceShape Dataset'	   /content/sample_data
 /content/face-shape-dataset.zip


In [ ]:
def padding(image):
  image=image
  h,w,_=image.shape
  if w/h<3/4:
    right=left=int(((3/4)*h-w )/2); bottom=top=0
  elif w/h>3/4:
    bottom=top=int(((4/3)*w-h )/2);left=right=0
  else: #w/h == 3/4:
    bottom=top=left=right=0
  return cv2.copyMakeBorder(image,top,bottom,left,right,cv2.BORDER_CONSTANT,0)

def crop_and_scale(input_path,output_path):
  mp_face_detection = mp.solutions.face_detection
  mp_drawing = mp.solutions.drawing_utils
  IMAGE_FILES = glob.glob(input_path)
  count=0
  with mp_face_detection.FaceDetection(model_selection=1, min_detection_confidence=0.5) as face_detection:
    for idx, file in enumerate(IMAGE_FILES):
      image = cv2.imread(file) #read

      none_num=0
      if image is None:        #nếu hình lỗi thì skip
        none_num=none_num+1
        print("NONE",none_num)
        continue

      h,w,_=image.shape        #shape
      results = face_detection.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))  #processing

      if results.detections is not None :
        count=count+1
        print("IMAGE %d TH: "%count,file)
        #cv2_imshow(image)
        ####   take values from mediapipe face dection/ bbox
        all_values=results.detections # label id + confidence + 4 points of bb + 6 keypoints
        #print(count,all_values)
        location_data=all_values[0].location_data  #take location from list (4 points of bb + 6 keypoints)
        bounding_box=location_data.relative_bounding_box # take bb from above list

        #take xmin, ymin, width, height
        xmin=bounding_box.xmin * w   ; ymin=bounding_box.ymin * h
        width=bounding_box.width * w ; height=bounding_box.height * h
        xmax = xmin + width          ; ymax = ymin + height

        # calculate CENTRE:
        #centre_x = xmin + (width/2); centre_y = ymin + (height/2)

        ###   NEW COORDINATE:   ###
        # change y: giữ cầm & trán
        xmin_n  = xmin; xmax_n = xmax
        ymin_n  = ymin -0.55*ymin  #trừ là lên
        ymax_n = ymax + 0.055*ymax
        width_n=(xmax_n-xmin_n); height_n=(ymax_n-ymin_n)

        # ratio w/h = 3/4:
        if (width_n == (3*height_n)/4) :
          xmin_n=xmin_n; xmax_n=xmax_n; ymin_n =ymin_n; ymax_n=ymax_n # remain all
        elif (width_n < (3*height_n)/4):
          delta= (3/4)*height_n - width_n
          xmax_n=xmax + delta/2; xmin_n=xmin - delta/2;  ymin_n =ymin_n; ymax_n=ymax_n # change x, remain y
          width_n= width_n + ((3*height_n)/4)
        else:
          delta = (4/3)*width_n - height_n
          xmin_n=xmin_n; xmax_n=xmax_n; ymin_n =ymin_n + delta/2; ymax_n=ymax_n - delta/2

        #CROP image:
        cropped_image=image[int(ymin_n):int(ymax_n),int(xmin_n):int(xmax_n)] #ymin: ymax, xmin:xmax

        #padding
        cropped_image=padding(cropped_image)

        # save image
        file_cropped = os.path.join(output_path,'%d.jpg'%count) #count=count+1
        cv2.imwrite(file_cropped, cropped_image)

        #print new ratio (w/h)
        h_n,w_n,_=cropped_image.shape
        print("ratio=",w_n/h_n)
        print("------------------------------------------------")
        #cv2_imshow(cropped_image)

In [ ]:
crop_and_scale(input_path="/content/FaceShape Dataset/testing_set/Heart/*.jpg",
               output_path='/content/drive/MyDrive/Inter_A2DS/New cropped/testing_set/Heart')

IMAGE 1 TH:  /content/FaceShape Dataset/testing_set/Heart/heart (912).jpg
ratio= 0.7475538160469667
------------------------------------------------
IMAGE 2 TH:  /content/FaceShape Dataset/testing_set/Heart/heart (3).jpg
ratio= 0.75
------------------------------------------------
IMAGE 3 TH:  /content/FaceShape Dataset/testing_set/Heart/heart (238).jpg
ratio= 0.7543859649122807
------------------------------------------------
IMAGE 4 TH:  /content/FaceShape Dataset/testing_set/Heart/heart (92).jpg
ratio= 0.7513812154696132
------------------------------------------------
IMAGE 5 TH:  /content/FaceShape Dataset/testing_set/Heart/heart (724).jpg
ratio= 0.7548076923076923
------------------------------------------------
IMAGE 6 TH:  /content/FaceShape Dataset/testing_set/Heart/heart (592).jpg
ratio= 0.7504332755632582
------------------------------------------------
IMAGE 7 TH:  /content/FaceShape Dataset/testing_set/Heart/heart (323).jpg
ratio= 0.7508038585209004
-----------------------